In [11]:
import codecs
import jieba
import jieba.analyse
import jieba.posseg
import pandas as pd
import re

import param
import util

##### 定义分词函数

In [2]:
def split_word(text, stopwords):
    word_list = jieba.cut(text)
    start = True
    result = ''
    for word in word_list:
        word = word.strip()
        if word not in stopwords:
            if start is True:
                result = word
                start = False
            else:
                result += ' ' + word
    return result.encode('utf-8')


##### 加载停用词

In [3]:
stopwords = {}
for line in codecs.open(param.data_path+'/input/stop.txt', 'r', 'utf-8'):
    stopwords[line.rstrip()] = 1
    

##### 加载数据 并分词

In [13]:
df_tr = []
for i, line in enumerate(open(param.data_path+'/input/train.txt', 'r')):
    if i % 1000 == 1:
        util.log('train data: iter = %d' % i)
#     line = line.replace(regex="[&]\w*[;]", value="")
    line = re.sub(re.compile('[&]\w*[;]'),'', line)
    segs = line.split('\t')
    if i ==1:
        print(segs[1]) # 看一下数据是否乱码
    row = {}
    row['id'] = segs[0]
    row['content'] = split_word(segs[1].strip(), stopwords)
    row['penalty'] = segs[2]
    row['laws'] = segs[3].strip()
    df_tr.append(row)
    if i == 5000: # 测试用
        break
    
df_tr = pd.DataFrame(df_tr)


df_te = []
for i, line in enumerate(open(param.data_path+'/input/test.txt', 'r')):
    if i % 1000 == 1:
        util.log('test data: iter = %d ' % i)
    segs = line.split('\t')
    row = {}
    row['id'] = segs[0]
    row['content'] = split_word(segs[1].strip(), stopwords)
    df_te.append(row)
    if i == 5000: # 测试用
        break
    
df_te = pd.DataFrame(df_te)

print(df_tr.shape)
print(df_tr.shape)
print(df_tr.info())
df_tr.head()

UnicodeDecodeError: 'gbk' codec can't decode byte 0xa6 in position 19: illegal multibyte sequence

##### 写出数据

In [8]:
df_all = pd.concat([df_tr, df_te]).fillna(0)
df_all.to_csv(param.data_path+'/output/corpus/all_data.csv', index=None)